In [2]:
# Isaac Berez
# 17.01.23
import sys
from scipy.io import mmread
import os
import glob
import pandas as pd
import numpy as np
from pandas_ods_reader import read_ods
from copy import deepcopy
import pprint
import json
import re
from datetime import datetime
import logging
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import HuberRegressor
from sklearn import preprocessing
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform
from sklearn.manifold import TSNE
from sklearn import metrics
from sklearn.cluster import DBSCAN
import seaborn as sns
from sklearn.neighbors import NearestNeighbors
from collections import Counter
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import dendrogram, linkage
import harmonypy as hm
from matplotlib.cm import ScalarMappable
from datetime import date
import mpld3
import hvplot.pandas
import holoviews as hv
from holoviews import opts
import panel as pn
import bokeh
from bokeh.resources import INLINE
from adjustText import adjust_text
from scipy.stats import mannwhitneyu, false_discovery_control, wilcoxon


import dimorph_processing as dp
import cell_comparison as cc
import sex_stats as ss

today = str(date.today())
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [4]:
gaba_folder = '/bigdata/isaac/gaba_files/'
#load in all filtered metadata with subclass labels
with open(gaba_folder + 'GABAmeta_data_df_plis_filtered_markers_2024-09-02.json') as json_data:
    gaba_metadata = json.load(json_data)
gaba_metadata = pd.DataFrame.from_dict(gaba_metadata, orient='columns')

In [7]:
len(np.unique(gaba_metadata.loc['cluster_label']))

50

In [11]:
output_folder = '/bigdata/isaac/gaba_files/sex_stats/volcano_plots/092324_sgr_norm_run/'

In [12]:
sig_genes_df = pd.read_csv(output_folder + 'sig_genes.csv',header=None)

In [13]:
sig_genes_df.rename(columns = {0:'id',1:'test',2:'gene',3:'delta',4:'-log10(p_adj)'}, 
            inplace = True)

In [14]:
sig_genes_df.head()

,id,test,gene,delta,-log10(p_adj)
0,2,ΔBN_m,0610012G03Rik,-1.359515,1.752553
1,2,ΔBN_m,1700020I14Rik,-1.225389,1.639768
2,2,ΔBN_m,1810037I17Rik,-1.049231,2.014789
3,2,ΔBN_m,2210013O21Rik,-1.424526,1.922974
4,2,ΔBN_m,2210016L21Rik,-1.312858,2.314693


In [32]:
np.where(sig_genes_df.loc[:,'test']== 'ΔBN_f')

(array([  686,   687,   688, ..., 40437, 40438, 40439]),)

In [33]:
np.where(sig_genes_df.loc[:,'test']== 'Δmf_B')

(array([  745,   746,   747, ..., 34063, 39900, 39901]),)

In [39]:
sig_genes_df.head()

,id,test,gene,delta,-log10(p_adj)
0,2,ΔBN_m,0610012G03Rik,-1.359515,1.752553
1,2,ΔBN_m,1700020I14Rik,-1.225389,1.639768
2,2,ΔBN_m,1810037I17Rik,-1.049231,2.014789
3,2,ΔBN_m,2210013O21Rik,-1.424526,1.922974
4,2,ΔBN_m,2210016L21Rik,-1.312858,2.314693


In [46]:
np.unique(sig_genes_df.loc[:,'test'])

array(['ΔBN_f', 'ΔBN_m', 'Δmf_B', 'Δmf_N'], dtype=object)

In [36]:
sig_genes_df.loc[:,'gene']

0        0610012G03Rik
1        1700020I14Rik
2        1810037I17Rik
3        2210013O21Rik
4        2210016L21Rik
             ...      
40435             Ppia
40436            Sumo2
40437             Tbca
40438            Tmed2
40439             Tpt1
Name: gene, Length: 40440, dtype: object

In [40]:
len(np.unique(sig_genes_df.loc[:,'id']))

43

In [9]:
columns = [item for item in sorted(np.unique(gaba_metadata.loc['cluster_label'])) for _ in range(8)] 

In [15]:
amit_sig_genes_df = pd.DataFrame(index=sig_genes_df.loc[:,'gene'], columns = columns)

In [17]:
output_folder

'/bigdata/isaac/gaba_files/sex_stats/volcano_plots/092324_sgr_norm_run/'

In [18]:
amit_sig_genes_df.to_csv(output_folder + 'amit_sig_genes_df.csv')

In [ ]:
file = open(output_folder + 'amit_sig_genes.csv', 'a+', newline ='')
# writing the data into the file
with file:    
    write = csv.writer(file)
    write.writerows(csv_row)

In [38]:
amit_sig_genes_df = pd.read_csv(output_folder + 'amit_sig_genes_df.csv', header = None)

/tmp/ipykernel_3613376/3325566429.py:1: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,2

In [51]:
amit_sig_genes_df

,0,1,2,3,4,5,6,7,8,9,...,391,392,393,394,395,396,397,398,399,400
0,gene,1,1,1,1,1,1,1,1,2,...,49,49,50,50,50,50,50,50,50,50
1,NaN,FC_BN_f,q_BN_f,FC_BN_m,q_BN_m,FC_mf_B,q_mf_B,FC_mf_N,q_mf_N,FC_BN_f,...,FC_mf_N,q_mf_N,FC_BN_f,q_BN_f,FC_BN_m,q_BN_m,FC_mf_B,q_mf_B,FC_mf_N,q_mf_N
2,0610012G03Rik,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1700020I14Rik,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1810037I17Rik,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40437,Ppia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40438,Sumo2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40439,Tbca,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40440,Tmed2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
amit_sig_genes_df.loc['0610012G03Rik',]

In [84]:
# Pivot the DataFrame to get delta and -log10(p_adj) in different columns for each gene
sig_genes_df_pivot_delta = sig_genes_df.pivot_table(index='gene', columns=['id', 'test'], values=['delta'])


In [85]:
sig_genes_df_pivot_delta

delta                                                            \
id               2                               3                              
test          ΔBN_f     ΔBN_m Δmf_B Δmf_N     ΔBN_f     ΔBN_m Δmf_B     Δmf_N   
gene                                                                            
0610007P14Rik   NaN       NaN   NaN   NaN       NaN       NaN   NaN       NaN   
0610009B22Rik   NaN       NaN   NaN   NaN       NaN       NaN   NaN       NaN   
0610010F05Rik   NaN       NaN   NaN   NaN       NaN       NaN   NaN       NaN   
0610011F06Rik   NaN       NaN   NaN   NaN       NaN       NaN   NaN       NaN   
0610012G03Rik   NaN -1.359515   NaN   NaN       NaN       NaN   NaN       NaN   
...             ...       ...   ...   ...       ...       ...   ...       ...   
mt-Nd3          NaN       NaN   NaN   NaN  1.713297  1.108662   NaN  1.431675   
mt-Nd4          NaN       NaN   NaN   NaN       NaN       NaN   NaN       NaN   
mt-Nd4l         NaN       NaN   NaN   NaN       NaN  1.466503   NaN       NaN   
mt-Nd5          NaN       NaN   NaN   NaN       NaN  1.699038   NaN       NaN   
mt-Nd6          NaN       NaN   NaN   NaN       NaN       NaN   NaN       NaN   

                                   ...                                        \
id                   5             ...        46                48        49   
test              ΔBN_f     ΔBN_m  ...     ΔBN_f ΔBN_m Δmf_N ΔBN_f     ΔBN_f   
gene                               ...                                         
0610007P14Rik       NaN       NaN  ...       NaN   NaN   NaN   NaN -2.228234   
0610009B22Rik       NaN       NaN  ...       NaN   NaN   NaN   NaN       NaN   
0610010F05Rik -3.138791 -2.744166  ...       NaN   NaN   NaN   NaN -2.815631   
0610011F06Rik       NaN       NaN  ... -2.817801   NaN   NaN   NaN       NaN   
0610012G03Rik       NaN -1.528088  ...       NaN   NaN   NaN   NaN -1.350169   
...                 ...       ...  ...       ...   ...   ...   ...       ...   
mt-Nd3              NaN       NaN  ...       NaN   NaN   NaN   NaN       NaN   
mt-Nd4              NaN       NaN  ...       NaN   NaN   NaN   NaN       NaN   
mt-Nd4l             NaN       NaN  ...       NaN   NaN   NaN   NaN       NaN   
mt-Nd5              NaN       NaN  ...       NaN   NaN   NaN   NaN       NaN   
mt-Nd6              NaN       NaN  ...       NaN   NaN   NaN   NaN       NaN   

                                                 
id                                 50            
test          ΔBN_m Δmf_B Δmf_N ΔBN_f     ΔBN_m  
gene                                             
0610007P14Rik   NaN   NaN   NaN   NaN       NaN  
0610009B22Rik   NaN   NaN   NaN   NaN       NaN  
0610010F05Rik   NaN   NaN   NaN   NaN       NaN  
0610011F06Rik   NaN   NaN   NaN   NaN       NaN  
0610012G03Rik   NaN   NaN   NaN   NaN -2.800377  
...             ...   ...   ...   ...       ...  
mt-Nd3          NaN   NaN   NaN   NaN       NaN  
mt-Nd4          NaN   NaN   NaN   NaN       NaN  
mt-Nd4l         NaN   NaN   NaN   NaN       NaN  
mt-Nd5          NaN   NaN   NaN   NaN       NaN  
mt-Nd6          NaN   NaN   NaN   NaN       NaN  

[5355 rows x 130 columns]

In [83]:
sig_genes_df.loc[sig_genes_df.loc[:,'gene'] == 'mt-Nd3',:]

,id,test,gene,delta,-log10(p_adj)
816,3,ΔBN_m,mt-Nd3,1.108662,4.000912
854,3,ΔBN_f,mt-Nd3,1.713297,3.693696
867,3,Δmf_N,mt-Nd3,1.431675,3.106671
5300,7,ΔBN_f,mt-Nd3,1.141092,inf
6636,10,ΔBN_m,mt-Nd3,1.180202,6.879097
8607,12,ΔBN_m,mt-Nd3,1.182591,8.096910
16318,21,Δmf_B,mt-Nd3,1.290419,20.186325
16482,21,Δmf_N,mt-Nd3,1.017935,17.273268
18375,23,Δmf_B,mt-Nd3,1.201756,1.940194
22637,29,ΔBN_f,mt-Nd3,1.017373,8.102373


In [86]:
# Pivot the DataFrame to get delta and -log10(p_adj) in different columns for each gene
sig_genes_df_pivot_p = sig_genes_df.pivot_table(index='gene', columns=['id', 'test'], values=['-log10(p_adj)'])


In [87]:
sig_genes_df_pivot_p

-log10(p_adj)                                                  \
id                       2                               3                    
test                  ΔBN_f     ΔBN_m Δmf_B Δmf_N     ΔBN_f     ΔBN_m Δmf_B   
gene                                                                          
0610007P14Rik           NaN       NaN   NaN   NaN       NaN       NaN   NaN   
0610009B22Rik           NaN       NaN   NaN   NaN       NaN       NaN   NaN   
0610010F05Rik           NaN       NaN   NaN   NaN       NaN       NaN   NaN   
0610011F06Rik           NaN       NaN   NaN   NaN       NaN       NaN   NaN   
0610012G03Rik           NaN  1.752553   NaN   NaN       NaN       NaN   NaN   
...                     ...       ...   ...   ...       ...       ...   ...   
mt-Nd3                  NaN       NaN   NaN   NaN  3.693696  4.000912   NaN   
mt-Nd4                  NaN       NaN   NaN   NaN       NaN       NaN   NaN   
mt-Nd4l                 NaN       NaN   NaN   NaN       NaN  2.813966   NaN   
mt-Nd5                  NaN       NaN   NaN   NaN       NaN  2.149631   NaN   
mt-Nd6                  NaN       NaN   NaN   NaN       NaN       NaN   NaN   

                                             ...                              \
id                             5             ...        46                48   
test              Δmf_N     ΔBN_f     ΔBN_m  ...     ΔBN_f ΔBN_m Δmf_N ΔBN_f   
gene                                         ...                               
0610007P14Rik       NaN       NaN       NaN  ...       NaN   NaN   NaN   NaN   
0610009B22Rik       NaN       NaN       NaN  ...       NaN   NaN   NaN   NaN   
0610010F05Rik       NaN  1.825192  1.742827  ...       NaN   NaN   NaN   NaN   
0610011F06Rik       NaN       NaN       NaN  ...  1.371059   NaN   NaN   NaN   
0610012G03Rik       NaN       NaN  2.472366  ...       NaN   NaN   NaN   NaN   
...                 ...       ...       ...  ...       ...   ...   ...   ...   
mt-Nd3         3.106671       NaN       NaN  ...       NaN   NaN   NaN   NaN   
mt-Nd4              NaN       NaN       NaN  ...       NaN   NaN   NaN   NaN   
mt-Nd4l             NaN       NaN       NaN  ...       NaN   NaN   NaN   NaN   
mt-Nd5              NaN       NaN       NaN  ...       NaN   NaN   NaN   NaN   
mt-Nd6              NaN       NaN       NaN  ...       NaN   NaN   NaN   NaN   

                                                           
id                   49                      50            
test              ΔBN_f ΔBN_m Δmf_B Δmf_N ΔBN_f     ΔBN_m  
gene                                                       
0610007P14Rik  1.501341   NaN   NaN   NaN   NaN       NaN  
0610009B22Rik       NaN   NaN   NaN   NaN   NaN       NaN  
0610010F05Rik  2.688549   NaN   NaN   NaN   NaN       NaN  
0610011F06Rik       NaN   NaN   NaN   NaN   NaN       NaN  
0610012G03Rik  3.348080   NaN   NaN   NaN   NaN  1.909368  
...                 ...   ...   ...   ...   ...       ...  
mt-Nd3              NaN   NaN   NaN   NaN   NaN       NaN  
mt-Nd4              NaN   NaN   NaN   NaN   NaN       NaN  
mt-Nd4l             NaN   NaN   NaN   NaN   NaN       NaN  
mt-Nd5              NaN   NaN   NaN   NaN   NaN       NaN  
mt-Nd6              NaN   NaN   NaN   NaN   NaN       NaN  

[5355 rows x 130 columns]

In [90]:
sig_genes_df_pivot_delta.index.equals(sig_genes_df_pivot_p.index), "DataFrames A and B must have matching indexes."

(True, 'DataFrames A and B must have matching indexes.')

In [93]:
# Interleave columns of A and B
new_columns = [col for pair in zip(sig_genes_df_pivot_delta.columns, sig_genes_df_pivot_p.columns) for col in pair]
sig_genes_df_pivot_combined = pd.concat([sig_genes_df_pivot_delta, sig_genes_df_pivot_p], axis=1)
sig_genes_df_pivot_combined = sig_genes_df_pivot_combined[new_columns]


In [94]:
sig_genes_df_pivot_combined

,delta,-log10(p_adj),delta,-log10(p_adj),delta,-log10(p_adj),delta,-log10(p_adj),delta,-log10(p_adj),...,delta,-log10(p_adj),delta,-log10(p_adj),delta,-log10(p_adj),delta,-log10(p_adj),delta,-log10(p_adj)
id,2,2,2,2,2,2,2,2,3,3,...,49,49,49,49,49,49,50,50,50,50
test,ΔBN_f,ΔBN_f,ΔBN_m,ΔBN_m,Δmf_B,Δmf_B,Δmf_N,Δmf_N,ΔBN_f,ΔBN_f,...,ΔBN_m,ΔBN_m,Δmf_B,Δmf_B,Δmf_N,Δmf_N,ΔBN_f,ΔBN_f,ΔBN_m,ΔBN_m
gene,,,,,,,,,,,,,,,,,,,,,
0610007P14Rik,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0610009B22Rik,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0610010F05Rik,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0610011F06Rik,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0610012G03Rik,NaN,NaN,-1.359515,1.752553,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.800377,1.909368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mt-Nd3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.713297,3.693696,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [98]:
output_folder

'/bigdata/isaac/gaba_files/sex_stats/volcano_plots/092324_sgr_norm_run/'

In [99]:
sig_genes_df_pivot_combined.to_csv(output_folder + 'sig_genes_formatted.csv')

In [97]:
sig_genes_df.loc[sig_genes_df.loc[:,'gene']=='mt-Nd3',:]

,id,test,gene,delta,-log10(p_adj)
816,3,ΔBN_m,mt-Nd3,1.108662,4.000912
854,3,ΔBN_f,mt-Nd3,1.713297,3.693696
867,3,Δmf_N,mt-Nd3,1.431675,3.106671
5300,7,ΔBN_f,mt-Nd3,1.141092,inf
6636,10,ΔBN_m,mt-Nd3,1.180202,6.879097
8607,12,ΔBN_m,mt-Nd3,1.182591,8.096910
16318,21,Δmf_B,mt-Nd3,1.290419,20.186325
16482,21,Δmf_N,mt-Nd3,1.017935,17.273268
18375,23,Δmf_B,mt-Nd3,1.201756,1.940194
22637,29,ΔBN_f,mt-Nd3,1.017373,8.102373


In [88]:
sig_genes_df

,id,test,gene,delta,-log10(p_adj)
0,2,ΔBN_m,0610012G03Rik,-1.359515,1.752553
1,2,ΔBN_m,1700020I14Rik,-1.225389,1.639768
2,2,ΔBN_m,1810037I17Rik,-1.049231,2.014789
3,2,ΔBN_m,2210013O21Rik,-1.424526,1.922974
4,2,ΔBN_m,2210016L21Rik,-1.312858,2.314693
...,...,...,...,...,...
40435,50,ΔBN_f,Ppia,-1.061595,1.669988
40436,50,ΔBN_f,Sumo2,-1.291682,3.039653
40437,50,ΔBN_f,Tbca,-1.716559,1.567411
40438,50,ΔBN_f,Tmed2,-4.023741,1.699981


,id,test,gene,delta,-log10(p_adj)
816,3,ΔBN_m,mt-Nd3,1.108662,4.000912
854,3,ΔBN_f,mt-Nd3,1.713297,3.693696
867,3,Δmf_N,mt-Nd3,1.431675,3.106671
5300,7,ΔBN_f,mt-Nd3,1.141092,inf
6636,10,ΔBN_m,mt-Nd3,1.180202,6.879097
8607,12,ΔBN_m,mt-Nd3,1.182591,8.096910
16318,21,Δmf_B,mt-Nd3,1.290419,20.186325
16482,21,Δmf_N,mt-Nd3,1.017935,17.273268
18375,23,Δmf_B,mt-Nd3,1.201756,1.940194
22637,29,ΔBN_f,mt-Nd3,1.017373,8.102373


In [18]:
tmp = sig_genes_df.groupby("id", group_keys=True)[['test','gene']].apply(lambda x: x)


In [22]:
tmp

test           gene
id                            
2  0      ΔBN_m  0610012G03Rik
   1      ΔBN_m  1700020I14Rik
   2      ΔBN_m  1810037I17Rik
   3      ΔBN_m  2210013O21Rik
   4      ΔBN_m  2210016L21Rik
...         ...            ...
50 40435  ΔBN_f           Ppia
   40436  ΔBN_f          Sumo2
   40437  ΔBN_f           Tbca
   40438  ΔBN_f          Tmed2
   40439  ΔBN_f           Tpt1

[40440 rows x 2 columns]

In [51]:
sig_genes_df.loc[:,'gene'].duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
40435     True
40436     True
40437     True
40438     True
40439     True
Name: gene, Length: 40440, dtype: bool

In [52]:
np.where(sig_genes_df.loc[:,'gene'] == 'Ppia')

(array([ 1582,  2249,  2497,  3133,  4435,  5655,  6365,  8181,  8493,
         8987, 10107, 23168, 23775, 25995, 27027, 29888, 31189, 32000,
        32487, 32678, 34243, 34751, 34907, 35281, 36567, 40234, 40435]),)

In [9]:
sig_genes_df.loc[sig_genes_df.loc[:,'gene'] == 'Vgf',:]

,id,test,gene,delta,-log10(p_adj)
11516,16,Δmf_B,Vgf,1.493220,1.762393
11598,16,Δmf_N,Vgf,2.170905,3.478465
13589,18,ΔBN_m,Vgf,5.571561,3.535818
13686,18,Δmf_B,Vgf,6.924836,7.616185
14281,19,ΔBN_m,Vgf,1.368141,1.350692
18064,22,Δmf_B,Vgf,1.895627,3.714959
18213,22,Δmf_N,Vgf,1.816315,2.457659
18343,23,ΔBN_m,Vgf,1.405360,3.787512
18347,23,ΔBN_f,Vgf,5.865146,1.534039
19924,24,Δmf_B,Vgf,3.145831,4.525889


In [45]:
ΔBN_m_df = sig_genes_df.loc[sig_genes_df.loc[:,'test'] == 'ΔBN_m',:]

In [46]:
ΔBN_m_df.head()

,id,test,gene,delta,-log10(p_adj)
0,2,ΔBN_m,0610012G03Rik,-1.359515,1.752553
1,2,ΔBN_m,1700020I14Rik,-1.225389,1.639768
2,2,ΔBN_m,1810037I17Rik,-1.049231,2.014789
3,2,ΔBN_m,2210013O21Rik,-1.424526,1.922974
4,2,ΔBN_m,2210016L21Rik,-1.312858,2.314693


In [49]:
#ΔBN_m_df.hvplot.scatter(x='delta', y='-log10(p_adj)', by='gene', 
                  #legend='top', height=400, width=400)

In [54]:
sig_genes_df.loc[sig_genes_df.loc[:,'test'] == 'ΔBN_f',:]

,id,test,gene,delta,-log10(p_adj)
686,2,ΔBN_f,2010107G23Rik,-3.113194,1.718808
687,2,ΔBN_f,2310036O22Rik,-2.051413,1.657058
688,2,ΔBN_f,2700094K13Rik,-1.619242,1.394457
689,2,ΔBN_f,Alyref,-2.708201,1.573191
690,2,ΔBN_f,Arl3,-1.803370,1.672695
...,...,...,...,...,...
40435,50,ΔBN_f,Ppia,-1.061595,1.669988
40436,50,ΔBN_f,Sumo2,-1.291682,3.039653
40437,50,ΔBN_f,Tbca,-1.716559,1.567411
40438,50,ΔBN_f,Tmed2,-4.023741,1.699981


In [ ]:
#split by test pairs
bn_sig_genes_df